# Exploratory Data Analysis

Nesse notebook, é conduzida uma análise exploratória de dados a fim de analisar a distribuição individual das variáveis e suas interrelações a fim de extrair insights para a etapa de modelagem preditiva. 

In [11]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

## importando dataset

In [12]:
datapath = Path('../data')

with open(f'{datapath}/data.pickle', 'rb') as file:
    data = pickle.load(file)

## Analisando variável resposta `cred_score`

Relembrando...

```bash
    1 = bom
    2 = ruim
```

In [15]:
data['cred_score'].value_counts()

1    700
2    300
Name: cred_score, dtype: int64